In [1]:
#
# 2017-10-05 - подготовка данных для анализа
#
#
#

In [2]:
from __future__ import division
import sys,os,datetime
import pandas as pd
from collections import Counter
import functools
#import tqdm
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics as skm
import pymorphy2, nltk

In [3]:
dftrain, dftest = pd.read_csv("../Data/train_task1_latest.csv"), pd.read_csv("../Data/test_task1_latest.csv")

In [4]:
dftrain.shape, dftest.shape, dftrain.columns.tolist()

((119398, 5),
 (74294, 4),
 ['paragraph_id', 'question_id', 'paragraph', 'question', 'target'])

In [53]:
print datetime.datetime.now()

def prelemming (ttext) :
    tttext  = ttext.decode('utf-8')
    xx = nltk.word_tokenize(tttext);
    xx = [mm2.parse(tt)[0].normal_form.encode('utf-8') for tt in xx]
    tttext = ' '.join(xx) 
    return(tttext)

def onlyX (ttext, notX={'PREP','CONJ','PRCL','INTJ'}) :
    tttext  = ttext.decode('utf-8') if len(ttext)>0 else ' '
    xx = nltk.word_tokenize(tttext);
    xx = [tt.encode('utf-8') for tt in xx if mm2.parse(tt)[0].tag.POS not in notX]
    ##print(ttext,xx)
    tttext = ' '.join(xx)
    return(tttext)

def notstop (ttext, stopwords) :
    tttext  = ttext.decode('utf-8')
    xx = nltk.word_tokenize(tttext);
    xx = [tt.encode('utf-8') for tt in xx if tt not in stopwords ]
    tttext = ' '.join(xx) 
    return(tttext)

stoprus = stoprus = nltk.corpus.stopwords.words('russian')

mm2 = pymorphy2.MorphAnalyzer()
for name, df, newfile in [('train', dftrain,'../Work/train_task1_lemma.csv'), ('test', dftest,'../Work/test_task1_lemma.csv')]:
    
    dfnew = df.copy()
    for index, row in dfnew.iterrows() : #tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):    
        if index%10000==0 : print '\t',datetime.datetime.now(), name, index
        parL  =  prelemming(row.paragraph)
        queL  =  prelemming(row.question)
        parLS =  notstop(parL,stoprus)
        queLS =  notstop(queL,stoprus)
        dfnew.loc[index, 'paragraphL']    = parL
        dfnew.loc[index, 'questionL']     = queL
        dfnew.loc[index, 'paragraphLS']   = parLS
        dfnew.loc[index, 'questionLS']    = queLS
        dfnew.loc[index, 'paragraphLX']   = onlyX(parL)
        dfnew.loc[index, 'questionLX']    = onlyX(queL)
        dfnew.loc[index, 'paragraphLSX']  = onlyX(parLS)
        dfnew.loc[index, 'questionLSX']   = onlyX(queLS)
        
    dfnew.to_csv(newfile,index=False)
    
print datetime.datetime.now()

2017-10-06 15:18:21.209574
	2017-10-06 15:18:21.330623 train 0
	2017-10-06 15:37:45.825226 train 10000
	2017-10-06 15:57:13.930638 train 20000
	2017-10-06 16:16:49.978047 train 30000
	2017-10-06 16:37:28.637745 train 40000
	2017-10-06 16:59:30.374890 train 50000
	2017-10-06 17:22:20.332750 train 60000
	2017-10-06 17:46:02.383362 train 70000
	2017-10-06 18:11:09.675297 train 80000
	2017-10-06 18:37:18.687798 train 90000
	2017-10-06 19:04:21.130113 train 100000
	2017-10-06 19:32:24.467346 train 110000
	2017-10-06 19:59:58.886025 test 0
	2017-10-06 20:17:50.915542 test 10000
	2017-10-06 20:36:52.412589 test 20000
	2017-10-06 20:57:44.436715 test 30000
	2017-10-06 21:18:50.088618 test 40000
	2017-10-06 21:40:57.847687 test 50000
	2017-10-06 22:04:32.849180 test 60000
	2017-10-06 22:29:40.339460 test 70000
2017-10-06 22:40:55.517930


In [5]:
stoprus = stoprus = nltk.corpus.stopwords.words('russian')

In [19]:
u'иметь' in stoprus, 'в' in stoprus, u'в' in stoprus

/home/boba/anaconda2/envs/tensorflow-gpu/lib/python2.7/site-packages/ipykernel/__main__.py:1: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


(False, False, True)

In [18]:
print len(stoprus)
#for tt in stoprus : print tt

151
